# 🌞 Objectives

- [upstage/SOLAR-10.7B-v1.0 ](https://huggingface.co/upstage/SOLAR-10.7B-v1.0)을 사례로 Transform 기반의 LLM 을 [kyujinpy/Open-platypus-Commercial](https://huggingface.co/datasets/kyujinpy/Open-platypus-Commercial) 으로 Intruction Tuning 해보기

- Transformer, LoRA 의 간략한 이해해보기

- Instruction Tuning 의 간략한 이해해보기

- Note
  ```
  1) Google Colab 을 Google Drive 를 마운트한 환경 기준으로 정리함
  2) Google Colab 에서 A100 40GB 필요 (Pro 이상 계정 구입 필요, 10$/month 수준)
  ```

# ⚾ pre-requistes

- [Google Colab](https://colab.research.google.com/) 와 A100 40GB 인스턴스  

- [Google Drive](https://drive.google.com/drive/) 와 30GB 정도 스토리지 용량

  - Google Colab 의 가상머신에 마운트 (자동으로 아래 마운트 될 것)

    ```
    /content/drive/MyDrive
    ```
- Hugging Face 계정

- `/content/drive/MyDrive/llm` 경로의 폴더 생성

- 파이선 패키지 설치

  - `huggingface_hub` 의 버전 충돌 발생시 `pip install --upgrade huggingface_hub` 등으로 업데이트

  ```bash
  pip install bitsandbytes==0.41.1
  pip install accelerate==0.21.0
  pip install appdirs
  pip install loralib
  pip install datasets
  pip install fire
  pip install git+https://github.com/huggingface/peft
  pip install transformers==4.34.1
  pip install sentencepiece sentence_transformers
  pip install scipy numpy scikit-learn pandas
  ```
- `/content/drive/MyDrive/llm` 에 Hugging Face 에서 모델과 데이터 다운로드 (아래 SSH 공개키를 Hugging Face 에 등록하는 과정 필요할 수 있음)

  ```bash
  git clone git@hf.co:upstage/SOLAR-10.7B-v1.0
  git clone git@hf.co:datasets/kyujinpy/Open-platypus-Commercial
  ```

  - Google Colab 의 가상머신에서 생성한 공개키를 Hugging Face 의 SSH Key 로 등록

    - Colab Console 에서 공개키 생성하여 SSH Agent 에 키 등록

      ```bash
      ssh-keygen -t ed25519 -C "your_email@example.com
      eval "$(ssh-agent -s)
      ssh-add ~/.ssh/id_ed25519
      ```
    
    - Huggig Face 의 Setting > SSH and GPH Key 메뉴에서 SSH Key 등록
      
      - 등록할 SSH Key 출력
      
        ```bash
        cat ~/.ssh/id_ed25519.pub
        ```

In [1]:
!pip install bitsandbytes==0.41.1
!pip install accelerate==0.21.0
!pip install appdirs
!pip install loralib
!pip install transformers==4.34.1
!pip install datasets==2.1
!pip install fire
!pip install git+https://github.com/huggingface/peft
!pip install sentencepiece sentence_transformers
!pip install scipy numpy scikit-learn pandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 16.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 53.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 99.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 38.7 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.20.2
    Uninstalling huggingface-hub-0.20.2:
      Successfully uninstalled huggingface-hub-0.20.2
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.15.0
    Uninstalling tokenizers-0.15.0:
      Successfully uninstalled tokenizers-0.15.0
  Attempting uninstall: transformers
    Found existing installation: transformers 4.35.2
    Uninstalling transformers-4.35.2:
      Successfully uninstalled transformers-4.35.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.4/325

## ⛳ (1) 파이선 설치 모듈 확인 및 임포트

In [2]:
import os
import os.path as osp
import sys
import fire
import json
from typing import List, Union

import torch
from torch.nn import functional as F

import transformers
from transformers import TrainerCallback, TrainingArguments, TrainerState, TrainerControl
from transformers.trainer_utils import PREFIX_CHECKPOINT_DIR
from transformers import LlamaForCausalLM, LlamaTokenizer
from transformers import AutoModelForCausalLM, AutoTokenizer

from datasets import load_dataset

from peft import (
    LoraConfig,
    get_peft_model,
    prepare_model_for_int8_training,
    set_peft_model_state_dict
)
from peft import PeftModel

## ⛳ (2) Pre-trained 모델 가지고와서 설정하기
- SOLAR-10.7B 는 AutoRegressive 방식인 Llama2 구조를 확장한 것임
- 모델 가중치는 8bit 로 초기 로드 (Quantization 과정 포함)
- 계산 과정에서는 float16 형식을 사용

In [3]:
device = 'auto'
# base_LLM_model = 'upstage/SOLAR-10.7B-v1.0'
base_LLM_model = '/content/drive/MyDrive/llm/SOLAR-10.7B-v1.0'

pre_model = AutoModelForCausalLM.from_pretrained(   # SOLAR-10.7B 는 CasualModeling, AR 방식
    base_LLM_model,                             # 모델 위치 (hugging face 로그인 사전에 해두어야 함)
    load_in_8bit=True,                          # LoRA 를 위해서 8 비트로 로딩
    torch_dtype=torch.float16,                  # 연산에 사용할 데이터는 float16
    device_map=device)                          # GPU, CPU 등은 알아서 선택

print('[Info] Model is downloaded & loaded !!!')

Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

[Info] Model is downloaded & loaded !!!



##⛳ (3) 토크나이저 정보 확인 및 설정

- PAD 는 오른쪽에 채우고 이 토큰을 편의상 0 으로 설정

- vocabulary 는 단어와 인덱스 형태로 구성되어 있고, 토크나이저가  문장을 단어 단위로 인덱싱하는데 사용


In [4]:
pre_tokenizer = AutoTokenizer.from_pretrained(base_LLM_model)

bos = pre_tokenizer.bos_token_id
eos = pre_tokenizer.eos_token_id
pad = pre_tokenizer.pad_token_id
pre_tokenizer.padding_side = "right"
print("[Info] BOS token:", bos)
print("[Info] EOS token:", eos)
print("[Info] PAD token:", pad)

if (pad == None) or (pad == eos):
    pre_tokenizer.pad_token_id = 0

print("[Info] vocabulary size:",pre_tokenizer.vocab_size)

[Info] BOS token: 1
[Info] EOS token: 2
[Info] PAD token: None
[Info] vocabulary size: 32000


### Tokenizer 동작 확인 및 vocabulary 형태 확인

- 이 tokenizer 는 `공백`을 `_` 로 대체하는 것을 볼 수 있음

- vcabuary 는 단어와 인덱스로 구성되어 있음

In [5]:
pre_tokenizer.encode('i am a boy')

[1, 613, 837, 264, 4531]

In [6]:
def find_key(val):
  for key, value in pre_tokenizer.vocab.items():
      if value == val:
          return key
  return None

find_key(1), find_key(613), find_key(837), find_key(4531)

('<s>', '▁i', '▁am', '▁boy')

In [7]:
pre_tokenizer.decode([1, 613, 837, 4531])

'<s> i am boy'

In [8]:
list(pre_tokenizer.vocab.keys())[0:5] , list(pre_tokenizer.vocab.values())[0:5]

(['validator', '▁gennaio', '模', 'ctor', 'avi'],
 [27201, 25540, 29266, 1810, 17101])

## ⛳ (4) Instruct Tuning 할 데이터 확보하기

- Instruct Tuning 은 주어진 문제를 어떻게 풀어야 할지 Instruction 을 제공하여 Pretrained 모델을 학습시킨다는 개념

- 여기서는 Instruct, Input, Response 의 형태로 프롬프트를 구성하고 프롬프트 상단에는 해당 형태가 이렇게 생겼음을 알려주는 문장을 추가해서 학습 세트를 위한 템플릿 만들었음

- 또한 템플릿에는 다운로드한 데이터에서 Intruction, Input 을 채워서 학습용 모델 입력 세트를 구성했음

### 다운로드한 데이터 로딩하고 확인해보기

- 학습 데이터는 input, ouput, instruction 이 주요 컬럼이고 input 이 비어있는 (길이가 0인 문자열) 데이터가 존재함

- 이것의 의미는 모델에 input 이 들어오면 output 처럼 대답을 해야 하고 ouput 처럼 문장을 만들때는 instruction 을 고려해야 한다는 의미임

- 또한 이 데이터는 MATH/PRM-800K, ARB 등 여러 오픈 데이터 등에서 가지고 온 것을 볼 수 있음

In [9]:
data = load_dataset('/content/drive/MyDrive/llm/Open-platypus-Commercial')

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Dataset parquet downloaded and prepared to /root/.cache/huggingface/datasets/parquet/Open-platypus-Commercial-012371d21695f2e4/0.0.0/0b6d5799bb726b24ad7fc7be720c170d8e497f575d02d47537de9a5bac074901. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [10]:
import pandas as pd
df = pd.DataFrame(data['train'])

df[df.apply(lambda x: len(x.input) > 0, axis=1)].head()

,input,output,instruction,data_source
12348,"Choose A, B, C or D as your solution.",The correct answer is $\mathrm{C}$. To answer ...,It is perhaps too easy to think of mathematica...,ARB
12349,"Choose A, B, C or D as your solution.","The correct answer is A. Although choices B, C...","""I acknowledge Shakespeare to be the world's g...",ARB
12350,"Choose A, B, C or D as your solution.","The correct answer is D. ""At present,"" says th...",The Shawnee people of the Ohio and Cumberland ...,ARB
12351,"Choose A, B, C or D as your solution.",The correct answer is D. Although the origin o...,\section{Passage $\mathrm{V}$}\nIn linguistics...,ARB
12352,"Choose A, B, C or D as your solution.",The correct answer is $\mathrm{C}$. This quest...,\section{Passage $\mathrm{V}$}\nEngineers and ...,ARB


In [11]:
df.data_source.value_counts()

MATH/PRM-800K      12298
airoboros           2605
leetcode_ne         1100
guanaco              797
ARB                  713
scibench             616
theoremqa            564
tigerbot-kaggle      386
Name: data_source, dtype: int64

### Instruct Tuning 에 쓸 템플릿과 템플릿 기반으로 학습용 입력 데이터를 만들기

- `instruct_template` 에 input 이 있는 경우와 없는 경우를 고려해서 prompt tempate 구조 정의

- 다운로드한 데이터를 가지고 `instruct_template` 기반으로 학습 데이터 구성할 때 쓸 `Prompter` 만들어 놓기

- 다운로드한 데이터, Prompter 를 가지고 토크나이징할 때 쓸 함수 만들어 놓기

  -

- 토크나이징 하기

#### 프롬프트 구조 정의하기

In [12]:
 instruct_template = {
    "prompt_input": '''Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n
                    ### Instruction:\n{instruction}\n\n
                    ### Input:\n{input}\n\n
                    ### Response:\n''',

    "prompt_no_input": '''Below is an instruction that describes a task. Write a response that appropriately completes the request.\n\n
                    ### Instruction:\n{instruction}\n\n
                    ### Response:\n''',

    "response_split": "### Response:"
}

#### 프롬프트 구조에 다운로드한 데이터를 집어넣을 유틸 구현하기

In [13]:
class Prompter(object):
    def __init__(self, verbose: bool = False):
        self.template = instruct_template

    def generate_prompt(
        self,
        instruction: str,
        input: Union[None, str] = None,
        label: Union[None, str] = None,
    ) -> str:

        if input: # input text가 있다면
            res = self.template["prompt_input"].format(
                instruction=instruction, input=input
            )
        else:
            res = self.template["prompt_no_input"].format(
                instruction=instruction
            )

        if label:
            res = f"{res}{label}"

        return res

    def get_response(self, output: str) -> str:
        return output.split(self.template["response_split"])[1].strip()

prompter = Prompter()

#### Pretrained model 과 함께 제공된 tokenizer 를 이용해서 모델 입력을 만드는 함수 구현하기

In [14]:
cutoff_len = 4096
train_on_inputs = False
add_eos_token = False

# Tokenizing
# - pre-trained 모델 로딩할 때 얻어온 tokenizer 를 이용 최대 입력 길이를 cutoff_len 으로 하고 padding 은 넣지 않도록 설정
# - EOS 토큰과 이건 신경 안써도 된다는 attension 마스크를 추가함
def tokenize(prompt, add_eos_token=True):
    # tokernizer 설정해서 가지고 오기 (cutoff_len 은 하이퍼파라미터 이용)
    result = pre_tokenizer( prompt, truncation=True, max_length=cutoff_len, padding=False, return_tensors=None,)

    # EOS 토큰 삽입해두고 Note 하기
    if (
        result["input_ids"][-1] != pre_tokenizer.eos_token_id   # EOS 토큰 없고
        and len(result["input_ids"]) < cutoff_len           # 길이가 cutoff_len 보다 작고
        and add_eos_token                                   # EOS 붙이기로 했으면
    ):
        # EOS 토큰을 붙이고 이 EOS 토큰에 집중(attention)하라고 1 로 설정
        result["input_ids"].append(pre_tokenizer.eos_token_id)
        result["attention_mask"].append(1)

    # 모델이 맞춰야 하는것이 입력임 (Auto Regressive 방식의 특징)
    result["labels"] = result["input_ids"].copy()

    return result

# 다운로드 받은 데이터를 Prompter 를 이용해서 구성하고
# tokenizer 를 이용해 pre-trained 모델과 함께 제공된 tokenizer 이용해서 입력 만듬
def generate_and_tokenize_prompt(data_point):
    # tokenizing 하기
    full_prompt = prompter.generate_prompt(data_point["instruction"], data_point["input"], data_point["output"])
    tokenized_full_prompt = tokenize(full_prompt)

    # input 이 없는 경우
    if not train_on_inputs:
        user_prompt = prompter.generate_prompt(data_point["instruction"], data_point["input"])
        tokenized_user_prompt = tokenize(user_prompt, add_eos_token=add_eos_token)
        user_prompt_len = len(tokenized_user_prompt["input_ids"])

        if add_eos_token:
            user_prompt_len -= 1

        # -100 으로 채워두어 학습할 때 무시하라고 마킹
        tokenized_full_prompt["labels"] = [-100] * user_prompt_len + tokenized_full_prompt["labels"][user_prompt_len:]

    return tokenized_full_prompt

#### 학습을 위한 모델 입력 만들기

In [15]:
data_shuffle = data["train"].shuffle()
train_data = data_shuffle.map(generate_and_tokenize_prompt)

  0%|          | 0/19079 [00:00<?, ?ex/s]

#### 학습용 모델 입력 확인해보기

In [16]:
df_data_shuffle = pd.DataFrame(data_shuffle)
df_data_shuffle.head()

,input,output,instruction,data_source
0,,"Note that $p(x) = 2x$ for $x = -3,$ 4, and 5, ...",Let $p(x)$ be a cubic polynomial such that $p(...,MATH/PRM-800K
1,,"First, I'll simplify the numerator by performi...",Express the following as a common fraction in ...,MATH/PRM-800K
2,,Let $AB = x$ and $AC = y$. Then we can write t...,Triangle $ABC$ is a right triangle with legs $...,MATH/PRM-800K
3,,Some of the consequences of climate change on ...,"BEGININPUT\nBEGINCONTEXT\ndate: August 5, 2021...",airoboros
4,,"Since $15$ and $6$ have a common factor of $3,...",Simplify $\frac{15}{6}.$,MATH/PRM-800K


In [17]:
df_train_data = pd.DataFrame(train_data)
df_train_data.head()

,input,output,instruction,data_source,input_ids,attention_mask,labels
0,,"Note that $p(x) = 2x$ for $x = -3,$ 4, and 5, ...",Let $p(x)$ be a cubic polynomial such that $p(...,MATH/PRM-800K,"[1, 20811, 349, 396, 13126, 369, 13966, 264, 3...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[-100, -100, -100, -100, -100, -100, -100, -10..."
1,,"First, I'll simplify the numerator by performi...",Express the following as a common fraction in ...,MATH/PRM-800K,"[1, 20811, 349, 396, 13126, 369, 13966, 264, 3...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[-100, -100, -100, -100, -100, -100, -100, -10..."
2,,Let $AB = x$ and $AC = y$. Then we can write t...,Triangle $ABC$ is a right triangle with legs $...,MATH/PRM-800K,"[1, 20811, 349, 396, 13126, 369, 13966, 264, 3...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[-100, -100, -100, -100, -100, -100, -100, -10..."
3,,Some of the consequences of climate change on ...,"BEGININPUT\nBEGINCONTEXT\ndate: August 5, 2021...",airoboros,"[1, 20811, 349, 396, 13126, 369, 13966, 264, 3...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[-100, -100, -100, -100, -100, -100, -100, -10..."
4,,"Since $15$ and $6$ have a common factor of $3,...",Simplify $\frac{15}{6}.$,MATH/PRM-800K,"[1, 20811, 349, 396, 13126, 369, 13966, 264, 3...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[-100, -100, -100, -100, -100, -100, -100, -10..."


## ⛳ (5) Instruct Tuning 위한 PEFT 방식을 LoRA로 정의하기

- Pretrained 가중치(모델)에 병렬적으로 Light Weight 한 행렬(모델)을 덧붙이는 방식이 LoRA 임.

  - 행렬(모델) 은 <Pretrained 모델의 입력 길이 x 사용자가 정의한 크기> 행렬과 <사용자가 정의한 크기> x <Pretrained 모델의 입력 길이 x 사용자가 정의한 크기> 의 두 개 행렬로 구성

  - Light Weight 한 수준은 결과 사용자가 정의하는 것임

    - 여기서는 lora_r 로 16으로 정의함

  - LoRA 로 튜닝할 때에는 입력이 Pretrained 모델과 요 행렬(모델)로 병렬로 주어지고, 출력시에는 두 모델의 출력이 덧셈이 됨. 다만, 이때 스케일링이 되는데 이것도 사용자가 정의하는 것이고 이는 행력 출력에 곱해짐

    - 여기서는 lora_alpha 로 16 으로 정의함

- 요 행렬을 LoRA Adpater 라고 하고 Pre-trained 모델의 어느 구성 요소에 병렬로 덧붙일 것인가를 결정해야 함

  - 여기서는 gate_proj, down_proj, up_proj 모듈 묶음에 적용하는 것으로 함.

  - 참고로 Pretrain 의 모델 구조는 아래처럼 출력됨

   ```bash
     (model): LlamaModel(
    (embed_tokens): Embedding(32000, 4096)
    (layers): ModuleList(
      (0-47): 48 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear8bitLt(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear8bitLt(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear8bitLt(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear8bitLt(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear8bitLt(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear8bitLt(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear8bitLt(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): LlamaRMSNorm()
  )
  (lm_head): Linear(in_features=4096, out_features=32000, bias=False)

   ```

#### LoRA 적용 방식 설정하기

In [18]:
lora_r = 16
lora_alpha = 16
lora_dropout = 0.05
lora_target_modules = ["gate_proj", "down_proj", "up_proj"]
config = LoraConfig(
    r=lora_r,
    lora_alpha=lora_alpha,
    target_modules=lora_target_modules,
    lora_dropout=lora_dropout,
    bias="none",
    task_type="CAUSAL_LM")

#### Pretrained 모델에 LoRA 적용

In [19]:
quant_model = prepare_model_for_int8_training(pre_model)
peft_model = get_peft_model(quant_model, config)

/usr/local/lib/python3.10/dist-packages/peft/utils/other.py:143: FutureWarning: prepare_model_for_int8_training is deprecated and will be removed in a future version. Use prepare_model_for_kbit_training instead.
  warnings.warn(


#### Out of Memory 발생으로 메모리 정리하기

In [22]:
quant_model = None
model = None

import gc
gc.collect()
torch.cuda.empty_cache()

## ⛳ (6) 학습 방석 설정하여 학습기 구성하기

- 배치 크기, 학습율, 학습율 변동 방식, 웜밍업 방식, 옵티마이저 등 설정하기. 이렇게 쓴다고 하면 그냥 따라 씀

- 이런 설정들로 transformer 의 Trainer 만듬

- `30` step 마다 체크포인트,  Google Drive 내 특정 위치(`/content/drive/MyDrive/llm/customized`)에 체크포인트 저장 (최대 2개)

- torch.fp16 으로 Pretrained 모델 로딩했으니 여기에 맞추었고 다른 것들은 다른것은 잘...

In [23]:
# 주요 하이퍼파라미터
batch_size = 16
num_epochs = 1
micro_batch = 1 # GPU 1개만 쓸거라서 나누지 않음
gradient_accumulation_steps = batch_size // micro_batch
lr_scheduler = 'cosine'
warmup_ratio = 0.06
learning_rate = 4e-4
optimizer = 'adamw_torch'
weight_decay = 0.01
max_grad_norm = 1.0

output_dir='/content/drive/MyDrive/llm/customized'
trainer = transformers.Trainer(
        model=peft_model,
        train_dataset=train_data,
        eval_dataset=None,
        args=transformers.TrainingArguments(
            per_device_train_batch_size = micro_batch,
            gradient_accumulation_steps = gradient_accumulation_steps,
            warmup_ratio=warmup_ratio,
            num_train_epochs=num_epochs,
            learning_rate=learning_rate,
            fp16=True,
            logging_steps=1,
            optim=optimizer,
            evaluation_strategy="no",
            save_strategy="steps",
            max_grad_norm = max_grad_norm,
            save_steps = 5,
            lr_scheduler_type=lr_scheduler,
            output_dir=output_dir,
            save_total_limit=2,
            load_best_model_at_end=False,
            ddp_find_unused_parameters=False,
            group_by_length = False
        ),
        data_collator=transformers.DataCollatorForSeq2Seq(
            pre_tokenizer, pad_to_multiple_of=8, return_tensors="pt", padding=True
        ),
    )

## ⛳ (7) 학습실행하기

- 학습 중간에 에러나서 특정 체크포인트부터 학습하도록 함

In [24]:
resume_from_checkpoint = '/content/drive/MyDrive/llm/customized/checkpoint-1170'
if resume_from_checkpoint:
    checkpoint_name = os.path.join(
        resume_from_checkpoint, "pytorch_model.bin"
    )  # All checkpoint

    if not os.path.exists(checkpoint_name):
        checkpoint_name = os.path.join(
            resume_from_checkpoint, "adapter_model.bin"
        )
        resume_from_checkpoint = (
            True
        )

    if os.path.exists(checkpoint_name):
        print(f"[Info] Restarting from {checkpoint_name}")
        adapters_weights = torch.load(checkpoint_name)
        set_peft_model_state_dict(peft_model, adapters_weights)
    else:
        print(f"[Error] Checkpoint {checkpoint_name} not found")

peft_model.config.use_cache = False

# 학습규모 파악해보기
print('[Info] Parameter volume to be trained')
peft_model.print_trainable_parameters()

# 학습하기
peft_model = torch.compile(peft_model)
torch.cuda.empty_cache()
trainer.train(resume_from_checkpoint=resume_from_checkpoint)

[Info] Restarting from /content/drive/MyDrive/llm/customized/checkpoint-1170/adapter_model.bin
[Info] Parameter volume to be trained
trainable params: 42,467,328 || all params: 10,773,991,424 || trainable%: 0.39416522928912257


You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/bitsandbytes/autograd/_functions.py:322: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


Step,Training Loss
1171,0.423900
1172,0.259700
1173,0.355500
1174,0.324400
1175,0.353400
1176,0.359100
1177,0.443600
1178,0.397400
1179,0.548500
1180,0.456900


TrainOutput(global_step=1192, training_loss=0.00779466085986003, metrics={'train_runtime': 821.8167, 'train_samples_per_second': 23.216, 'train_steps_per_second': 1.45, 'total_flos': 5.786190877267722e+17, 'train_loss': 0.00779466085986003, 'epoch': 1.0})

## ⛳ (8) 학습 완료된 것 저장하기

In [26]:
peft_model.save_pretrained(output_dir)
model_path = os.path.join(output_dir, "pytorch_model.bin")
torch.save({}, model_path)
pre_tokenizer.save_pretrained(output_dir)

('/content/drive/MyDrive/llm/customized/tokenizer_config.json',
 '/content/drive/MyDrive/llm/customized/special_tokens_map.json',
 '/content/drive/MyDrive/llm/customized/tokenizer.model',
 '/content/drive/MyDrive/llm/customized/added_tokens.json',
 '/content/drive/MyDrive/llm/customized/tokenizer.json')

In [27]:
print(peft_model)

OptimizedModule(
  (_orig_mod): PeftModelForCausalLM(
    (base_model): LoraModel(
      (model): LlamaForCausalLM(
        (model): LlamaModel(
          (embed_tokens): Embedding(32000, 4096)
          (layers): ModuleList(
            (0-47): 48 x LlamaDecoderLayer(
              (self_attn): LlamaAttention(
                (q_proj): Linear8bitLt(in_features=4096, out_features=4096, bias=False)
                (k_proj): Linear8bitLt(in_features=4096, out_features=1024, bias=False)
                (v_proj): Linear8bitLt(in_features=4096, out_features=1024, bias=False)
                (o_proj): Linear8bitLt(in_features=4096, out_features=4096, bias=False)
                (rotary_emb): LlamaRotaryEmbedding()
              )
              (mlp): LlamaMLP(
                (gate_proj): lora.Linear8bitLt(
                  (base_layer): Linear8bitLt(in_features=4096, out_features=14336, bias=False)
                  (lora_dropout): ModuleDict(
                    (default): Dropout(p=0.05,

## ⛳ (9) 학습 완료된 것을 Pretrained 모델에 통합하기

##### Pretrained 베이스 모델과 Lora 수행한 모델 합치기
- 학습 완료 직후 여러가지 gc 를 해줘도 메모리 점유율 잘 않떨어짐. 인스턴스 새로 받아서 하는 것이 좋음


In [1]:
!pip install bitsandbytes==0.41.1
!pip install accelerate==0.21.0
!pip install appdirs
!pip install loralib
!pip install transformers==4.34.1
!pip install datasets==2.1
!pip install fire
!pip install git+https://github.com/huggingface/peft
!pip install sentencepiece sentence_transformers
!pip install scipy numpy scikit-learn pandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 17.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 58.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 96.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 33.4 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.20.2
    Uninstalling huggingface-hub-0.20.2:
      Successfully uninstalled huggingface-hub-0.20.2
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.15.0
    Uninstalling tokenizers-0.15.0:
      Successfully uninstalled tokenizers-0.15.0
  Attempting uninstall: transformers
    Found existing installation: transformers 4.35.2
    Uninstalling transformers-4.35.2:
      Successfully uninstalled transformers-4.35.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.4/325

In [ ]:
import os
import os.path as osp
import sys
import fire
import json
from typing import List, Union

import torch
from torch.nn import functional as F

import transformers
from transformers import TrainerCallback, TrainingArguments, TrainerState, TrainerControl
from transformers.trainer_utils import PREFIX_CHECKPOINT_DIR
from transformers import LlamaForCausalLM, LlamaTokenizer
from transformers import AutoModelForCausalLM, AutoTokenizer

from datasets import load_dataset

from peft import (
    LoraConfig,
    get_peft_model,
    prepare_model_for_int8_training,
    set_peft_model_state_dict
)
from peft import PeftModel

# 인스턴스 새로 만들어서 하는 것이 좋음. gc 해줘도 메모리 부족함
base_model = AutoModelForCausalLM.from_pretrained(
    '/content/drive/MyDrive/llm/SOLAR-10.7B-v1.0',
    return_dict = True,
    torch_dtype=torch.float16,
    device_map='auto')


In [8]:
pre_tokenizer = AutoTokenizer.from_pretrained('/content/drive/MyDrive/llm/SOLAR-10.7B-v1.0')

In [5]:
peft_model = PeftModel.from_pretrained(base_model, '/content/drive/MyDrive/llm/customized', 'auto')
fin_model = peft_model.merge_and_unload()

#### 합친거 파일로 저장하기

In [ ]:
final_save_folder = '/content/drive/MyDrive/llm/custom_final'
fin_model.save_pretrained(final_save_folder)
model_path = os.path.join(final_save_folder, "pytorch_model.bin")
torch.save({}, model_path)
pre_tokenizer.save_pretrained(final_save_folder)

# (10) 최종 Instruction Tuning 모델 동작 확인하기

* 메모리 부족할 수 있으니 새로 인스턴스 만들어서 다시 모델 로딩부터 → 40GB 로는 로라랑 베이스모델 못 불러옴

* Run Pod 로 옮김

In [7]:
!pip install bitsandbytes==0.41.1
!pip install accelerate==0.21.0
!pip install appdirs
!pip install loralib
!pip install transformers==4.34.1
!pip install datasets==2.1
!pip install fire
!pip install git+https://github.com/huggingface/peft
!pip install sentencepiece sentence_transformers
!pip install scipy numpy scikit-learn pandas

  Cloning https://github.com/huggingface/peft to /tmp/pip-req-build-1hepiu94
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/peft /tmp/pip-req-build-1hepiu94
  Resolved https://github.com/huggingface/peft to commit bf54136a79cc85b0e4c3915b4e1eb158f43c4b73
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [22]:
!pip install --upgrade accelerate

In [27]:

from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

# 모델과 토크나이저 불러오기
device = 'cuda:0'
custom_model_path = '/content/drive/MyDrive/llm/custom_final'  # 커스터마이즈된 모델 경로

# 커스터마이즈된 모델 로드
custom_model = AutoModelForCausalLM.from_pretrained(
    custom_model_path,
    # load_in_8bit=True,  # 이 옵션은 커스터마이즈된 모델에 따라 조정 필요
    # torch_dtype=torch.float16,  # 마찬가지로, 모델에 따라 조정 필요
    # device_map=device  # GPU, CPU 등은 알아서 선택
)
tokenizer = AutoTokenizer.from_pretrained(custom_model_path)

print('[Info] Customized model is downloaded & loaded !!!')

# 모델을 평가 모드로 설정
custom_model.eval()

# GPU 사용 가능한 경우 GPU에 모델 로드
if torch.cuda.is_available():
    custom_model = custom_model.cuda()

# 입력 텍스트
input_text = "please translate 'i am a boy' into korean"

# 토크나이즈 하고 텐서로 변환
inputs = tokenizer(input_text, return_tensors="pt")
if torch.cuda.is_available():
    inputs = {k: v.cuda() for k, v in inputs.items()}

# 모델 실행하여 출력 얻기
with torch.no_grad():
    outputs = custom_model(**inputs)

# 출력 처리 (예: 텍스트로 변환)
output_text = tokenizer.decode(outputs.logits[0], skip_special_tokens=True)

print(output_text)

Some weights of LlamaForCausalLM were not initialized from the model checkpoint at /content/drive/MyDrive/llm/custom_final and are newly initialized: ['layers.16.mlp.gate_proj.weight', 'layers.8.self_attn.v_proj.weight', 'layers.0.input_layernorm.weight', 'layers.9.self_attn.o_proj.weight', 'layers.19.post_attention_layernorm.weight', 'layers.13.post_attention_layernorm.weight', 'lm_head.weight', 'layers.40.post_attention_layernorm.weight', 'layers.15.self_attn.q_proj.weight', 'layers.17.mlp.down_proj.weight', 'layers.29.self_attn.q_proj.weight', 'layers.33.self_attn.v_proj.weight', 'layers.32.input_layernorm.weight', 'layers.37.self_attn.o_proj.weight', 'layers.0.self_attn.o_proj.weight', 'layers.6.mlp.down_proj.weight', 'layers.27.mlp.up_proj.weight', 'layers.9.mlp.gate_proj.weight', 'layers.37.input_layernorm.weight', 'layers.2.self_attn.v_proj.weight', 'layers.37.self_attn.q_proj.weight', 'layers.19.mlp.up_proj.weight', 'layers.31.input_layernorm.weight', 'layers.40.mlp.down_proj.w

[Info] Customized model is downloaded & loaded !!!


OutOfMemoryError: CUDA out of memory. Tried to allocate 224.00 MiB. GPU 0 has a total capacty of 39.56 GiB of which 18.81 MiB is free. Process 15709 has 39.54 GiB memory in use. Of the allocated memory 39.03 GiB is allocated by PyTorch, and 20.38 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [26]:
import gc
del custom_model
del tokenizer
gc.collect()
torch.cuda.empty_cache()